In [9]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env('LANGCHAIN_TRACING_V2')
_set_env('LANGCHAIN_ENDPOINT')
os.environ["LANGCHAIN_PROJECT"] = "problem_decomposition"

In [ ]:
# Test config file
import os
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--mode', type=str, default='train')
parser.add_argument('--data_file', type=str)
parser.add_argument('--glove_word_file', type=str, default=glove_word_file)
parser.add_argument('--save', type=str, default='HOTPOT')

config = parser.parse_args(args=[])


# Test run

In [ ]:
# Graph structure
from graph_builder import build_graph
ExperimentName = 'test'
graph = build_graph(ExperimentName)

# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Prep data
from datasets import load_dataset

train_set = load_dataset("hotpot_qa", 'distractor', split='train', trust_remote_code=True)
dev_set = load_dataset("hotpot_qa", 'distractor', split='validation', trust_remote_code=True)

In [ ]:
data = train_set[0]

# Extract data info
id = data['id']
question = data['question']
answer = data['answer']
context = [
    "{}: {}".format(title, " ".join(sentences)) 
    for title, sentences in zip(data['context']['title'], data['context']['sentences'])
]

# Invoke the graphs
thread = {"configurable": {"thread_id": id}}
prompt = {
    'messages': question,
    'context': context
}
response = graph.invoke(prompt, thread)

In [21]:
thread = {"configurable": {"thread_id": 22}}
prompt = {
    'messages': 'hi',
    'context': 'hi'
}
response = graph.invoke(prompt, thread)

In [22]:
graph.get_state(thread).values['messages']

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='a5ad1ae9-6266-4c89-857c-0e8d606090ab'),
 AIMessage(content='To address the original problem effectively, we need to follow a structured approach by synthesizing the solutions from the sub-problems:\n\n1. **Identify the core objective and constraints**:\n   - The core objective is to verify, refine, and break down the given problem.\n   - Constraints may include time limitations, resource availability, and any specific requirements provided.\n\n2. **Check if the problem is well-defined with clear success criteria**:\n   - Ensure that the problem statement is clear, concise, and includes measurable success criteria to determine when the problem is solved.\n\n3. **Highlight key assumptions and prerequisites**:\n   - Identify any assumptions made during the analysis process and list out the prerequisites needed to solve the problem effectively.\n\n4. **Reformulate if needed to make more precise and actionable**:\n 

In [ ]:
for m in response['messages']:
    m.pretty_print()

# Validation